# Yelp API Webscraping

In [12]:
import pandas as pd 
import json
import requests
import time

# Compiled Code
- Don't forget to comment so we can understand!

## known issues:
- function populates dataframe with NaN rows
- do we want business name?
- businesses have multiple categories and function only picks up one at a time
    - currently the 'alias' portion picks up the alias of the business (name), not the category alias

In [31]:
# NOTHING IN THIS CELL NEEDS TO GET CHANGED 
# IF YOU HAVE YOUR API KEY IN 'creds.json' IN ./Assets

# format your json file as a dictionary containing api key with DOUBLE QUOTES
# {"api": "your_super_long_api_key"}
creds_file = open('../Assets/creds.json')

# load credentials into variable
yelp_credentials = json.loads(creds_file.read())
api_key = yelp_credentials['api']
headers = {'Authorization': 'Bearer %s' % api_key}

# this is the url we use to make broad business searches
# https://www.yelp.com/developers/documentation/v3/business_search

url = 'https://api.yelp.com/v3/businesses/search'

In [21]:
params = {'term':'food', 'location': 'Los Angeles','limit': 50, 'offset'}

In [22]:
# ONLY RUN THIS CELL IF YOU WANT TO MAKE A REQUEST
req = requests.get(url, params=params, headers=headers)
print(f'Status Code: {req.status_code}')

Status Code: 200


In [36]:
def get_businesses(location, term, api_key):
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = 'https://api.yelp.com/v3/businesses/search'

    data = []
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': location.replace(' ', '+'),
            'term': term.replace(' ', '+'),
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    data = pd.DataFrame(data)
    return data

In [37]:
get_businesses('Los Angeles','food',api_key)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,mcHP3IfI1b7bXJZKD6PLHA,angry-chickz-los-angeles-2,Angry Chickz,https://s3-media3.fl.yelpcdn.com/bphoto/jC89o6...,False,https://www.yelp.com/biz/angry-chickz-los-ange...,752,"[{'alias': 'southern', 'title': 'Southern'}, {...",4.5,"{'latitude': 34.102039, 'longitude': -118.299858}",[],$$,"{'address1': '5065 Hollywood Blvd', 'address2'...",+13239282825,(323) 928-2825,4915.161843
1,pjh40JY5YwWeV8aKhkXERg,myungrang-hot-dog-california-market-la-los-ang...,Myungrang Hot Dog-California Market LA,https://s3-media2.fl.yelpcdn.com/bphoto/RyKaa2...,False,https://www.yelp.com/biz/myungrang-hot-dog-cal...,703,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}]",4.5,"{'latitude': 34.06577, 'longitude': -118.30847}",[],$,"{'address1': '450 S Western Ave', 'address2': ...",+12133757518,(213) 375-7518,1282.173440
2,6QeZEUhFk1_vZhWvktKnmA,fat-sals-deli-hollywood-los-angeles,Fat Sal's Deli - Hollywood,https://s3-media4.fl.yelpcdn.com/bphoto/8WkYWo...,False,https://www.yelp.com/biz/fat-sals-deli-hollywo...,2017,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 34.0945448886537, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '1300 N Highland Ave', 'address2'...",+13236739446,(323) 673-9446,3977.528150
3,b4SH4SbQUJfXxh6hNkF0wg,eggslut-los-angeles-7,Eggslut,https://s3-media2.fl.yelpcdn.com/bphoto/TvaHjA...,False,https://www.yelp.com/biz/eggslut-los-angeles-7...,5597,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 34.0505289, 'longitude': -118.248...",[],$,"{'address1': '317 S Broadway', 'address2': '',...",+12136250292,(213) 625-0292,6796.488502
4,k8xtuersBm2cZl0Lao2K5g,the-bun-shop-los-angeles,The Bun Shop,https://s3-media4.fl.yelpcdn.com/bphoto/BmWALC...,False,https://www.yelp.com/biz/the-bun-shop-los-ange...,1506,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"{'latitude': 34.0740309655666, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '151 N Western Ave', 'address2': ...",+13234681031,(323) 468-1031,1817.156287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NIfvfVLbXGlGJOm4tmHwvg,lemonade-los-angeles-9,Lemonade,https://s3-media1.fl.yelpcdn.com/bphoto/OTzvVO...,False,https://www.yelp.com/biz/lemonade-los-angeles-...,808,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",3.5,"{'latitude': 33.9411074693798, 'longitude': -1...",[],$$,"{'address1': '500 World Way', 'address2': '', ...",+14247445978,(424) 744-5978,15424.505155
996,1xe7ikU6Tr4Om5fGqRWsHg,el-colmao-los-angeles,El Colmao,https://s3-media1.fl.yelpcdn.com/bphoto/tUNJqM...,False,https://www.yelp.com/biz/el-colmao-los-angeles...,234,"[{'alias': 'cuban', 'title': 'Cuban'}]",4.0,"{'latitude': 34.04693, 'longitude': -118.2873865}",[],$$,"{'address1': '2328 W Pico Blvd', 'address2': '...",+12133866131,(213) 386-6131,3530.561521
997,u2LSp-abP0yjZfhZWxfPpw,cha-cha-matcha-west-hollywood-3,Cha Cha Matcha,https://s3-media2.fl.yelpcdn.com/bphoto/GYuMbk...,False,https://www.yelp.com/biz/cha-cha-matcha-west-h...,150,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,"{'latitude': 34.0804007872686, 'longitude': -1...",[],$$,"{'address1': '510 N Robertson Blvd', 'address2...",,,6219.920718
998,RpgMXd-usckXwHZZM3f7Qg,jim-dandy-fried-chicken-los-angeles,Jim Dandy Fried Chicken,https://s3-media3.fl.yelpcdn.com/bphoto/SGcynA...,False,https://www.yelp.com/biz/jim-dandy-fried-chick...,227,"[{'alias': 'southern', 'title': 'Southern'}, {...",4.0,"{'latitude': 33.93159, 'longitude': -118.29123}",[],$,"{'address1': '11328 S Vermont Ave', 'address2'...",+13237795567,(323) 779-5567,14722.784446


In [24]:
businesses_object = yelp['businesses']
businesses_object[15]

# id
# coordinates
# price
# review_count
# rating
# location > zip_code
# location > city
# categories > alias

{'id': 'PXYKRPkEcf4Mczfp_AfP0w',
 'alias': 'corn-man-los-angeles-3',
 'name': 'Corn Man',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/4L7s-o9rvdT7xz-6tNDlkg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/corn-man-los-angeles-3?adjust_creative=1hBkN0UhEY8hcT3qXV3ivQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=1hBkN0UhEY8hcT3qXV3ivQ',
 'review_count': 902,
 'categories': [{'alias': 'foodstands', 'title': 'Food Stands'},
  {'alias': 'mexican', 'title': 'Mexican'}],
 'rating': 4.5,
 'coordinates': {'latitude': 34.0725774690603, 'longitude': -118.214694509262},
 'transactions': [],
 'price': '$',
 'location': {'address1': '2338 Workman St',
  'address2': '',
  'address3': '',
  'city': 'Los Angeles',
  'zip_code': '90031',
  'country': 'US',
  'state': 'CA',
  'display_address': ['2338 Workman St', 'Los Angeles, CA 90031']},
 'phone': '+12137882323',
 'display_phone': '(213) 788-2323',
 'distance': 9902.711468013575}

In [29]:
# stole this function from project 3 - jerry

def get_price(yelp_object):
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = 'https://api.yelp.com/v3/businesses/search'
    businesses = []
    # empty list container containing dictionaries representing each unique business
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': location.replace(' ', '+'),
            'term': term.replace(' ', '+'),
            'offset': offset
        }
    businesses_object = yelp_object['businesses']
    response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            # looking at each subreddit which is passed into our function
            for i, business in enumerate(list(businesses_object)):
                # each dictionary will contain all of the desired information from each post
                business_dict = {}
                if 'price' in businesses_object[i].keys():
                    business_dict['id']           = business['id']
                    business_dict['latitude']     = business['coordinates']['latitude']
                    business_dict['longitude']    = business['coordinates']['longitude']
                    business_dict['price']        = business['price']
                    business_dict['review_count'] = business['review_count']
                    business_dict['rating']       = business['rating']
                    business_dict['zip_code']     = business['location']['zip_code']
                    business_dict['city']         = business['location']['city']
                    business_dict['alias']        = business['alias']

                    # populate the posts list with each post dictionary
                businesses.append(business_dict)
            businesses = pd.DataFrame(businesses)

        # just a little sanity check to see how far along our function is going
        #print(f'{i+1} out of {len(businesses_object)}')
    return businesses
    #businesses = pd.DataFrame(businesses)
   
    

In [30]:
get_price(yelp)

,id,latitude,longitude,price,review_count,rating,zip_code,city,alias
0,CcqraT0cuGKYEcZ1ri_kxg,34.040403,-118.253512,$$,689.0,5.0,90015,Los Angeles,broken-mouth-lees-homestyle-los-angeles-5
1,mcHP3IfI1b7bXJZKD6PLHA,34.102039,-118.299858,$$,752.0,4.5,90027,Los Angeles,angry-chickz-los-angeles-2
2,pjh40JY5YwWeV8aKhkXERg,34.065770,-118.308470,$,703.0,4.5,90020,Los Angeles,myungrang-hot-dog-california-market-la-los-ang...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b4SH4SbQUJfXxh6hNkF0wg,34.050529,-118.248619,$,5597.0,4.0,90013,Los Angeles,eggslut-los-angeles-7
5,DrToq9357afdpOyO5w0Y5w,34.064009,-118.300701,$$,1961.0,4.5,90020,Los Angeles,yup-dduk-la-los-angeles
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,omaJSGvnj2vaIJ_MBxeyBw,34.032251,-118.334644,$,96.0,4.5,90018,Los Angeles,l-a-birria-los-angeles
8,WjuO8PhtqINeWqm-ei8NQA,34.069300,-118.292530,$$,508.0,4.5,90020,Los Angeles,dumpling-house-los-angeles-2
9,PQGsaaJ9YVL_Sgiy7aB70Q,34.063440,-118.298310,$,223.0,4.0,90020,Los Angeles,street-food-of-seoul-los-angeles


In [54]:
df = pd.DataFrame(business_list)

In [56]:
df

,id,latitude,longitude,price,review_count,rating,zip_code,city,alias
0,CcqraT0cuGKYEcZ1ri_kxg,34.040403,-118.253512,$$,689.0,5.0,90015,Los Angeles,broken-mouth-lees-homestyle-los-angeles-5
1,KQBGm5G8IDkE8LeNY45mbA,34.045605,-118.236061,$$,8205.0,4.0,90013,Los Angeles,wurstküche-los-angeles-2
2,pjh40JY5YwWeV8aKhkXERg,34.065770,-118.308470,$,702.0,4.5,90020,Los Angeles,myungrang-hot-dog-california-market-la-los-ang...
3,b4SH4SbQUJfXxh6hNkF0wg,34.050529,-118.248619,$,5597.0,4.0,90013,Los Angeles,eggslut-los-angeles-7
4,sYn3SNQP-j2t2XSwjlCbRg,34.064690,-118.308760,$$,1618.0,4.5,90020,Los Angeles,montys-good-burger-los-angeles
...,...,...,...,...,...,...,...,...,...
6095,CbW8U0QAwh5XRkaLt0xNZA,34.077585,-118.259599,$$,4023.0,4.0,90026,Los Angeles,masa-of-echo-park-los-angeles
6096,fjWpZPr4YDJRKdnj0fqlkg,34.083320,-118.327380,$$,33.0,5.0,90038,Los Angeles,omakase-teriyaki-los-angeles
6097,ohosmz6FXVAeoW5nUkYwng,34.090596,-118.277172,$$,2151.0,4.5,90026,Los Angeles,pine-and-crane-los-angeles
6098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df.to_csv('../data/echo1023.csv')

## aerika section

In [1]:
hi

NameError: name 'hi' is not defined

In [2]:
my

NameError: name 'my' is not defined

In [3]:
name

NameError: name 'name' is not defined

## echo section

added comment

In [ ]:
added cell

## jerry section

In [ ]:
# ds;alkfjds;lfkas;dghas'dlfjas;dlkfjas'dfj

In [1]:
# this is another change a;sdfja;dfkasd;fajsd;fasdkfjas;fwef

In [2]:
# yet another change ;asdlfkjas;dknva;djaweoifsd;laknfas;dfknasd

In [3]:
x = np.exp(52)
x

3.831008000716577e+22